In [1]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
df = pd.read_csv('player_mvp_stats.csv')
df.head()

,Unnamed: 0,player,pos,age,tm,g,gs,mp,fg,fga,...,pts_max,share,team,w,l,w/l%,gb,ps/g,pa/g,srs
0,0,a.c. green,pf,27,lal,82,21,26.4,3.1,6.6,...,0.0,0.0,los angeles lakers,58.0,24.0,0.707,5.0,106.3,99.6,6.73
1,1,byron scott,sg,29,lal,82,82,32.1,6.1,12.8,...,0.0,0.0,los angeles lakers,58.0,24.0,0.707,5.0,106.3,99.6,6.73
2,2,elden campbell,pf,22,lal,52,0,7.3,1.1,2.4,...,0.0,0.0,los angeles lakers,58.0,24.0,0.707,5.0,106.3,99.6,6.73
3,3,irving thomas,pf,25,lal,26,0,4.2,0.7,1.9,...,0.0,0.0,los angeles lakers,58.0,24.0,0.707,5.0,106.3,99.6,6.73
4,4,james worthy,sf,29,lal,78,74,38.6,9.2,18.7,...,0.0,0.0,los angeles lakers,58.0,24.0,0.707,5.0,106.3,99.6,6.73


In [3]:
df = df.fillna(0)
del df['Unnamed: 0']

In [4]:
predictors = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg%', '3p',
       '3pa', '3p%', '2p', '2pa', '2p%', 'efg%', 'ft', 'fta', 'ft%', 'orb',
       'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'year', 'w', 'l', 'w/l%', 'gb', 'ps/g',
       'pa/g', 'srs']

In [5]:
train = df[df['year'] < 2020]
test = df[df['year'] == 2020]

rdg = Ridge(alpha=.1)
rdg.fit(train[predictors], train['share'])

Ridge(alpha=0.1)

In [6]:
predictions = rdg.predict(test[predictors])

predictions = pd.DataFrame(predictions, columns=['predictions'], index=test.index)

In [7]:
predictions

,predictions
596,0.019887
597,-0.028127
598,-0.014142
599,0.008575
600,0.029770
...,...
12627,0.004633
12628,0.038976
12629,0.007462
12630,0.009776


In [14]:
combination = pd.concat([test[['player', 'share', 'year']], predictions], axis=1)
combination = combination.sort_values('share', ascending=False)
combination['rk'] = list(range(1, len(combination) + 1))

In [16]:
combination = combination.sort_values('predictions', ascending=False)
combination['predicted_rk'] = list(range(1, len(combination) + 1))

In [19]:
np.sqrt(mean_squared_error(combination['share'], combination['predictions']))

0.0503376846609708

In [9]:
mean_squared_error(combination['share'], combination['predictions'])

0.0025338824970273345

In [17]:
combination

,player,share,year,predictions,rk,predicted_rk
11857,giannis antetokounmpo,0.952,2020,0.241064,1,1
9197,james harden,0.363,2020,0.189782,3,2
8763,luka dončić,0.198,2020,0.170165,4,3
3462,lebron james,0.746,2020,0.161245,2,4
3450,anthony davis,0.081,2020,0.146607,6,5
...,...,...,...,...,...,...
3865,darius garland,0.000,2020,-0.046235,487,524
9201,p.j. tucker,0.000,2020,-0.050043,33,525
1261,terrance ferguson,0.000,2020,-0.054012,381,526
7075,zach collins,0.000,2020,-0.059261,442,527


In [18]:
def find_ap(df):
    actual = combination.sort_values('share', ascending=False).head(5)
    predicted = combination.sort_values('predictions', ascending=False)
    ps = []
    found = 0
    seen = 1
    
    for index, row in predicted.iterrows():
        if row['player'] in actual['player'].values:
            found += 1
            ps.append(found/seen)
        seen += 1
    return sum(ps) / len(ps)

In [19]:
find_ap(combination)

0.9428571428571428

In [20]:
years = list(range(1991, 2021))

In [48]:
aps = []
all_prediction = []

for year in years[5:]:
    train = df[df['year'] < year ]
    test = df[df['year'] == year]
    rdg.fit(train[predictors], train['share'])
    y_pred = rdg.predict(test[predictors])
    y_pred = pd.DataFrame(y_pred, columns=['predictions'], index=test.index)
    combination = pd.concat([test[['player', 'share', 'year']], y_pred], axis=1)
    all_prediction.append(combination)
    aps.append(find_ap(combination))

In [49]:
sum(aps) / len(aps)

0.7110422205931348

In [50]:
combination 

,player,share,year,predictions
596,aaron gordon,0.0,2020,0.019887
597,al-farouq aminu,0.0,2020,-0.028127
598,b.j. johnson,0.0,2020,-0.014142
599,d.j. augustin,0.0,2020,0.008575
600,evan fournier,0.0,2020,0.029770
...,...,...,...,...
12627,rondae hollis-jefferson,0.0,2020,0.004633
12628,serge ibaka,0.0,2020,0.038976
12629,shamorie ponds,0.0,2020,0.007462
12630,stanley johnson,0.0,2020,0.009776


In [45]:
aps

[0.39584675975653416,
 0.7733333333333333,
 0.386993006993007,
 0.8392857142857142,
 0.9266666666666665,
 0.6815270935960591,
 0.9666666666666666,
 0.6928571428571428,
 0.5644444444444445,
 0.4003208556149732,
 0.6676190476190477,
 0.9428571428571428,
 0.75,
 0.8253968253968254,
 0.7253968253968253,
 0.4323717948717949,
 0.6976190476190476,
 0.6327838827838828,
 0.9266666666666665,
 0.8333333333333333,
 0.7611111111111111,
 0.7909090909090909,
 0.6375757575757576,
 0.9428571428571428]

In [36]:
def add_rank(combination):
    combination = combination.sort_values('predictions', ascending=False)
    combination['predicted_rk'] = list(range(1, len(combination) + 1))
    combination = combination.sort_values('share', ascending=False)
    combination['rk'] = list(range(1, len(combination) + 1))
    combination['diff'] = combination['rk'] - combination['predicted_rk']
    return combination

In [47]:
ranking = add_rank(all_prediction[1])
ranking[ranking['rk'] < 6].sort_values('diff', ascending=False)

,player,share,predictions,predicted_rk,rk,diff
1475,karl malone,0.857,0.195454,2,1,-1
10041,michael jordan,0.832,0.172254,3,2,-1
851,grant hill,0.327,0.130560,6,3,-3
4379,tim hardaway,0.207,0.060021,19,4,-15
7496,glen rice,0.117,0.035304,49,5,-44


In [145]:
def backtest(df, model, year, predictors):
    aps = []
    all_predictions = []

    for year in years[5:]:
        train = df[df['year'] < year ]
        test = df[df['year'] == year]
        model.fit(train[predictors], train['share'])
        y_pred = model.predict(test[predictors])
        y_pred = pd.DataFrame(y_pred, columns=['predictions'], index=test.index)
        new_df = pd.concat([test[['player', 'share']], y_pred], axis=1)
        new_df = add_rank(new_df)
        all_predictions.append(new_df)
        aps.append(find_ap(new_df))
        
    return sum(aps)/len(aps), aps, pd.concat(all_prediction)
#     return pd.concat(all_predictions)

In [150]:
mean_ap, aps, all_pred = backtest(df, rdg, years[5:], predictors)

In [152]:
aps

[0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428,
 0.9428571428571428]

In [172]:
def backtest(df, model, year, predictors):
    aps = []
    all_prediction = []

    for year in years[5:]:
        train = df[df['year'] < year ]
        test = df[df['year'] == year]
        rdg.fit(train[predictors], train['share'])
        y_pred = rdg.predict(test[predictors])
        y_pred = pd.DataFrame(y_pred, columns=['predictions'], index=test.index)
        combination = pd.concat([test[['player', 'share']], y_pred], axis=1)
        all_prediction.append(combination)
        aps.append(find_ap(combination))

#     return sum(aps)/len(aps), aps, pd.concat(all_prediction)
    return train

In [173]:
aps = backtest(df, rdg, years[5:], predictors)

In [176]:
aps['year'].unique()

array([1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2017, 2010, 2011, 2012, 2013, 2014, 2015, 2008, 2009,
       2016, 2018, 2019, 2004, 2005, 2006, 2007], dtype=int64)

In [157]:
aps = []
all_prediction = []

for year in years[5:]:
    train = df[df['year'] < year ]
    test = df[df['year'] == year]
    rdg.fit(train[predictors], train['share'])
    y_pred = rdg.predict(test[predictors])
    y_pred = pd.DataFrame(y_pred, columns=['predictions'], index=test.index)
    combination = pd.concat([test[['player', 'share']], y_pred], axis=1)
    all_prediction.append(combination)
    aps.append(find_ap(combination))

In [138]:
sum(aps)/len(aps)

0.7110422205931348